In [2]:
import pandas as pd
import yfinance as yf
import re
import datetime
import time
import os
import logging
import random
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s',
                   handlers=[logging.FileHandler("stock_data.log"), logging.StreamHandler()])
logger = logging.getLogger(__name__)

def format_ticker_symbol(ticker, max_retries=3):
    """Format ticker symbol for Yahoo Finance with improved detection and retry logic."""
    if "." in ticker: return ticker
    
    for attempt in range(max_retries):
        try:
            for exchange in [{'suffix': '.NS', 'name': 'NSE'}, {'suffix': '.BO', 'name': 'BSE'}]:
                formatted_ticker = f"{ticker}{exchange['suffix']}"
                try:
                    test_data = yf.download(formatted_ticker, period="1d", progress=False, timeout=5)
                    if not test_data.empty:
                        logger.info(f"Found valid {exchange['name']} ticker: {formatted_ticker}")
                        return formatted_ticker
                except Exception:
                    # Continue to next exchange
                    pass
            
            try:
                if not yf.download(ticker, period="1d", progress=False, timeout=5).empty:
                    logger.info(f"Found valid ticker without exchange suffix: {ticker}")
                    return ticker
            except Exception:
                # Continue to retry
                pass
            
            if attempt < max_retries - 1:
                # Exponential backoff with jitter
                wait_time = (2 ** attempt) + random.uniform(0, 1)
                logger.warning(f"Ticker lookup failed, retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
        except Exception as e:
            logger.error(f"Error in ticker format attempt {attempt+1}: {str(e)}")
            if attempt < max_retries - 1:
                wait_time = (2 ** attempt) + random.uniform(0, 1)
                time.sleep(wait_time)
    
    logger.warning(f"Could not find valid exchange for {ticker}, using as is")
    return ticker

def get_nearest_trading_day(ticker, target_date, max_retries=3):
    """Find the nearest trading day to the target date with retry logic."""
    formatted_ticker = format_ticker_symbol(ticker)
    target_date = pd.to_datetime(target_date)
    
    for window_days in [15, 30, 60, 90]:
        for attempt in range(max_retries):
            try:
                start_date = target_date - datetime.timedelta(days=window_days)
                end_date = target_date + datetime.timedelta(days=window_days)
                
                stock_data = yf.download(formatted_ticker, start=start_date.strftime('%Y-%m-%d'),
                                        end=end_date.strftime('%Y-%m-%d'), progress=False, timeout=10)
                
                if not stock_data.empty:
                    stock_data['date_diff'] = abs((stock_data.index - target_date).days)
                    return stock_data['date_diff'].idxmin().strftime('%Y-%m-%d')
                
                if attempt < max_retries - 1:
                    # Exponential backoff with jitter
                    wait_time = (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"Data fetch failed for {ticker}, retrying in {wait_time:.2f} seconds...")
                    time.sleep(wait_time)
            except Exception as e:
                logger.warning(f"Error finding trading day with ±{window_days} days (attempt {attempt+1}): {str(e)}")
                if attempt < max_retries - 1:
                    wait_time = (2 ** attempt) + random.uniform(0, 1)
                    time.sleep(wait_time)
    
    return None

def get_stock_price(ticker, date, max_retries=3):
    """Get stock price for a ticker on a specific date with retry mechanism."""
    for attempt in range(max_retries):
        try:
            formatted_ticker = format_ticker_symbol(ticker)
            trading_date = get_nearest_trading_day(formatted_ticker, date)
            
            # Try alternate exchange if needed
            if not trading_date:
                alt_ticker = formatted_ticker.replace(".NS", ".BO") if ".NS" in formatted_ticker else formatted_ticker.replace(".BO", ".NS") if ".BO" in formatted_ticker else None
                if alt_ticker:
                    trading_date = get_nearest_trading_day(alt_ticker, date)
                    if trading_date: formatted_ticker = alt_ticker
            
            if not trading_date:
                return None, None
            
            for inner_attempt in range(max_retries):
                try:
                    stock_data = yf.download(formatted_ticker, start=trading_date,
                                          end=pd.to_datetime(trading_date) + datetime.timedelta(days=1),
                                          progress=False, timeout=15)
                    
                    if stock_data.empty:
                        if inner_attempt < max_retries - 1:
                            wait_time = (2 ** inner_attempt) + random.uniform(0, 1)
                            time.sleep(wait_time)
                            continue
                        return None, None
                    
                    closing_price = stock_data['Close'].iloc[0]
                    if isinstance(closing_price, pd.Series):
                        closing_price = closing_price.iloc[0]
                        
                    return closing_price, trading_date
                except Exception as e:
                    logger.warning(f"Error downloading price data (inner attempt {inner_attempt+1}): {str(e)}")
                    if inner_attempt < max_retries - 1:
                        wait_time = (2 ** inner_attempt) + random.uniform(1, 2)
                        time.sleep(wait_time)
            
            # If we reach here, all inner attempts failed
            if attempt < max_retries - 1:
                wait_time = (2 ** attempt) + random.uniform(1, 3)
                logger.warning(f"All download attempts failed, retrying entire process in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
        except Exception as e:
            logger.error(f"Error getting stock price for {ticker} (attempt {attempt+1}): {str(e)}")
            if attempt < max_retries - 1:
                wait_time = (2 ** attempt) + random.uniform(1, 3)
                time.sleep(wait_time)
    
    return None, None

def find_value_by_patterns(df, patterns, year_column):
    """Find values using regex patterns."""
    for pattern in patterns:
        matching_rows = df.index[df.index.str.lower().str.contains(pattern, regex=True)]
        if len(matching_rows) > 0:
            return df.loc[matching_rows[0], year_column], pattern
    return None, None

def get_financial_data(ticker, year, max_retries=3):
    """Get financial data for a company for a specific year with retry mechanism."""
    financial_data = {'total_assets': None, 'total_liabilities': None, 'outstanding_shares': None, 
                     'ebit': None, 'data_date': None}
    
    for attempt in range(max_retries):
        try:
            formatted_ticker = format_ticker_symbol(ticker)
            stock = yf.Ticker(formatted_ticker)
            
            # Get financial statement data
            try:
                balance_sheet = stock.get_balance_sheet()
                income_stmt = stock.get_income_stmt()
                quarterly_balance_sheet = stock.quarterly_balance_sheet
                quarterly_income_stmt = stock.quarterly_income_stmt
                stock_info = stock.info
                
                if balance_sheet is None and quarterly_balance_sheet is None:
                    if attempt < max_retries - 1:
                        wait_time = (2 ** attempt) + random.uniform(1, 3)
                        logger.warning(f"No financial data for {ticker}, retrying in {wait_time:.2f} seconds...")
                        time.sleep(wait_time)
                        continue
            except Exception as e:
                logger.warning(f"Error getting financial statements (attempt {attempt+1}): {str(e)}")
                if attempt < max_retries - 1:
                    wait_time = (2 ** attempt) + random.uniform(1, 3)
                    time.sleep(wait_time)
                    continue
            
            # Process balance sheet data
            for sheet_type, sheet in [('annual', balance_sheet), ('quarterly', quarterly_balance_sheet)]:
                if financial_data['total_assets'] is not None:
                    break
                    
                if sheet is not None and not sheet.empty:
                    year_columns = [col for col in sheet.columns if pd.to_datetime(col).year == year]
                    if year_columns:
                        year_column = year_columns[0] if sheet_type == 'annual' else sorted(year_columns)[-1]
                        financial_data['data_date'] = year_column.strftime('%Y-%m-%d')
                        
                        # Find total assets and liabilities
                        financial_data['total_assets'], _ = find_value_by_patterns(sheet, 
                            [r'total.*assets', r'assets.*total', r'assets'], year_column)
                        
                        financial_data['total_liabilities'], _ = find_value_by_patterns(sheet, 
                            [r'total.*liabilities', r'liabilities.*total', r'total debt'], year_column)
                        
                        # Find outstanding shares from various sources
                        shares_patterns = [r'shares outstanding', r'outstanding shares', r'common shares outstanding']
                        financial_data['outstanding_shares'], _ = find_value_by_patterns(sheet, shares_patterns, year_column)
                        
                        # Calculate liabilities from assets and equity if needed
                        if financial_data['total_liabilities'] is None and financial_data['total_assets'] is not None:
                            total_equity, _ = find_value_by_patterns(sheet, 
                                [r'total.*equity', r'equity.*total', r'stockholders.*equity'], year_column)
                            
                            if total_equity is not None:
                                financial_data['total_liabilities'] = financial_data['total_assets'] - total_equity
            
            # Try getting shares from info if not found
            if financial_data['outstanding_shares'] is None and stock_info is not None:
                for key in ['sharesOutstanding', 'impliedSharesOutstanding', 'floatShares']:
                    if key in stock_info and stock_info[key] is not None and stock_info[key] > 1000:
                        financial_data['outstanding_shares'] = stock_info[key]
                        break
            
            # Get EBIT data
            for sheet_type, sheet in [('annual', income_stmt), ('quarterly', quarterly_income_stmt)]:
                if financial_data['ebit'] is not None:
                    break
                    
                if sheet is not None and not sheet.empty:
                    year_columns = [col for col in sheet.columns if pd.to_datetime(col).year == year]
                    if year_columns:
                        year_column = year_columns[0] if sheet_type == 'annual' else sorted(year_columns)[-1]
                        ebit_patterns = [r'^ebit$', r'earnings before interest and tax', r'operating income']
                        avoid_patterns = [r'ebitda', r'depreciation', r'amortization']
                        
                        # First look for direct EBIT entry
                        for pattern in ebit_patterns:
                            matching_rows = sheet.index[sheet.index.str.lower().str.contains(pattern, regex=True)]
                            if len(matching_rows) > 0:
                                row_name = matching_rows[0].lower()
                                if not any(re.search(avoid, row_name) for avoid in avoid_patterns):
                                    financial_data['ebit'] = sheet.loc[matching_rows[0], year_column]
                                    break
                        
                        # If EBIT not found directly, try operating profit or calculate it
                        if financial_data['ebit'] is None:
                            try:
                                financial_data['ebit'], _ = find_value_by_patterns(sheet, 
                                    [r'operating profit', r'operating income'], year_column)
                                
                                if financial_data['ebit'] is None:
                                    gross_profit, _ = find_value_by_patterns(sheet, [r'gross profit'], year_column)
                                    sga_expense, _ = find_value_by_patterns(sheet, [r'selling.*general.*administrative'], year_column) 
                                    
                                    if gross_profit is not None:
                                        financial_data['ebit'] = gross_profit - (sga_expense or 0)
                            except Exception:
                                pass
            
            # If we got some data, break the retry loop
            if any(value is not None for value in financial_data.values()):
                break
            
            # If we didn't get any data, retry
            if attempt < max_retries - 1:
                wait_time = (2 ** attempt) + random.uniform(1, 3)
                logger.warning(f"No financial data for {ticker}, retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                
        except Exception as e:
            logger.error(f"Error getting financial data for {ticker} (attempt {attempt+1}): {str(e)}")
            if attempt < max_retries - 1:
                wait_time = (2 ** attempt) + random.uniform(1, 3)
                time.sleep(wait_time)
    
    return financial_data

def setup_excel_writer(file_path, columns):
    """Create and set up Excel file with formatted headers."""
    if os.path.exists(file_path):
        os.remove(file_path)
        
    writer = pd.ExcelWriter(file_path, engine='openpyxl')
    pd.DataFrame(columns=columns).to_excel(writer, sheet_name='Stock Data', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Stock Data']
    
    # Format headers
    header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
    border = Border(left=Side(border_style="thin"), right=Side(border_style="thin"),
                   top=Side(border_style="thin"), bottom=Side(border_style="thin"))
    
    for col_num, column_title in enumerate(columns, 1):
        cell = worksheet.cell(row=1, column=col_num)
        cell.font = Font(bold=True, color="FFFFFF")
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center')
        cell.border = border
        
        column_letter = get_column_letter(col_num)
        worksheet.column_dimensions[column_letter].width = min(len(column_title) + 5, 30)
    
    writer.close()
    return True

def write_row_to_excel(file_path, row_data, row_num):
    """Write a single row of data to Excel file."""
    try:
        workbook = load_workbook(file_path)
        worksheet = workbook['Stock Data']
        
        border = Border(left=Side(border_style="thin"), right=Side(border_style="thin"),
                       top=Side(border_style="thin"), bottom=Side(border_style="thin"))
        
        for col_num, value in enumerate(row_data.values(), 1):
            cell = worksheet.cell(row=row_num, column=col_num, value=value)
            cell.border = border
            cell.alignment = Alignment(horizontal='right' if isinstance(value, (int, float)) else 'left')
        
        workbook.save(file_path)
        return True
    except Exception as e:
        logger.error(f"Error writing row to Excel: {str(e)}")
        return False

def process_companies(input_file, output_file):
    """Process companies with real-time Excel updates and improved rate limiting."""
    logger.info(f"Starting processing of companies from {input_file}")
    
    try:
        # Read input file
        df = pd.read_excel(input_file)
        required_columns = ['SYMBOL', 'NAME OF COMPANY', 'INDUSTRY']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"Input file missing required columns")
            return False
        
        # Set up Excel file
        columns = ['Symbol', 'Company', 'Industry', 'Year', 'Stock Date', 'Stock Price',
                  'Total Assets', 'Total Liabilities', 'Outstanding Shares', 'EBIT', 'Financial Data Date']
        setup_excel_writer(output_file, columns)
        
        # Process each company
        years = [2021, 2022, 2023, 2024]
        total_companies = len(df)
        current_row = 2  # Start after header
        
        for index, row in df.iterrows():
            ticker, company_name, industry = row['SYMBOL'], row['NAME OF COMPANY'], row['INDUSTRY']
            logger.info(f"Processing {index+1}/{total_companies}: {company_name} ({ticker})")
            
            for year in years:
                try:
                    # Adaptive retry delay based on position in the dataset
                    # More backoff for later items to spread out requests
                    base_delay = 1 + (index / total_companies)
                    
                    # Get stock price and financial data with exponential backoff
                    stock_price, actual_date = get_stock_price(ticker, f"{year}-09-01")
                    
                    # Add additional delay between major API calls
                    time.sleep(base_delay + random.uniform(0, 1))
                    
                    financial_data = get_financial_data(ticker, year)
                    
                    # Create row data
                    row_data = {
                        'Symbol': ticker, 'Company': company_name, 'Industry': industry, 'Year': year,
                        'Stock Date': actual_date, 'Stock Price': stock_price,
                        'Total Assets': financial_data['total_assets'],
                        'Total Liabilities': financial_data['total_liabilities'],
                        'Outstanding Shares': financial_data['outstanding_shares'],
                        'EBIT': financial_data['ebit'],
                        'Financial Data Date': financial_data['data_date']
                    }
                    
                    # Write data directly to Excel
                    write_row_to_excel(output_file, row_data, current_row)
                    current_row += 1
                    
                    # Adaptive rate limiting
                    wait_time = base_delay + random.uniform(0.5, 2.5)
                    logger.info(f"Waiting {wait_time:.2f} seconds before next request...")
                    time.sleep(wait_time)
                    
                except Exception as e:
                    logger.error(f"Error processing {ticker} for {year}: {str(e)}")
                    # Write empty row to maintain consistency
                    empty_row = {
                        'Symbol': ticker, 'Company': company_name, 'Industry': industry, 'Year': year,
                        'Stock Date': None, 'Stock Price': None, 'Total Assets': None,
                        'Total Liabilities': None, 'Outstanding Shares': None, 'EBIT': None,
                        'Financial Data Date': None
                    }
                    write_row_to_excel(output_file, empty_row, current_row)
                    current_row += 1
                    time.sleep(1 + random.uniform(0, 1))  # Still wait between errors
        
        logger.info(f"Processing completed. Results saved to {output_file}")
        return True
    except Exception as e:
        logger.error(f"Error processing companies: {str(e)}")
        return False

def calculate_metrics(input_file, output_file):
    """Calculate and write financial metrics."""
    logger.info(f"Calculating financial metrics from {input_file}")
    
    try:
        # Read input data and set up output file
        input_df = pd.read_excel(input_file, sheet_name='Stock Data')
        columns = list(input_df.columns) + ['Market Cap', 'Book Value', 'Enterprise Value', 'Price to Book', 'EV to EBIT']
        setup_excel_writer(output_file, columns)
        
        # Process each row individually for memory efficiency
        current_row = 2  # Start after header
        for _, row in input_df.iterrows():
            # Calculate metrics
            market_cap = row['Stock Price'] * row['Outstanding Shares'] if not (pd.isna(row['Stock Price']) or pd.isna(row['Outstanding Shares'])) else None
            book_value = row['Total Assets'] - row['Total Liabilities'] if not (pd.isna(row['Total Assets']) or pd.isna(row['Total Liabilities'])) else None
            enterprise_value = (market_cap + row['Total Liabilities']) if not (pd.isna(market_cap) or pd.isna(row['Total Liabilities'])) else None
            
            price_to_book = market_cap / book_value if not (pd.isna(market_cap) or pd.isna(book_value) or book_value == 0) else None
            ev_to_ebit = enterprise_value / row['EBIT'] if not (pd.isna(enterprise_value) or pd.isna(row['EBIT']) or row['EBIT'] == 0) else None
            
            # Add metrics to row data
            output_row = row.to_dict()
            output_row.update({
                'Market Cap': market_cap,
                'Book Value': book_value,
                'Enterprise Value': enterprise_value,
                'Price to Book': price_to_book,
                'EV to EBIT': ev_to_ebit
            })
            
            # Write row to Excel
            write_row_to_excel(output_file, output_row, current_row)
            current_row += 1
        
        logger.info(f"Metrics calculation completed. Results saved to {output_file}")
        return True
    except Exception as e:
        logger.error(f"Error calculating metrics: {str(e)}")
        return False

def main():
    """Main function to run the stock data collection and analysis."""
    logger.info("Starting stock data collection program")
    
    try:
        input_file = 'name.xlsx'
        output_file = 'stock_financial_data.xlsx'
        metrics_file = 'financial_metrics.xlsx'
        
        if not os.path.exists(input_file):
            logger.error(f"Input file {input_file} does not exist")
            return 1
        
        # Process companies from input file with real-time Excel updates
        if process_companies(input_file, output_file):
            # Calculate financial metrics
            calculate_metrics(output_file, metrics_file)
            logger.info("Stock data collection and analysis completed successfully")
            return 0
        return 1
    except Exception as e:
        logger.error(f"Error in main process: {str(e)}")
        return 1

if __name__ == "__main__":
    main()

2025-04-17 08:58:58,567 - INFO - Starting stock data collection program
2025-04-17 08:58:58,567 - INFO - Starting processing of companies from name.xlsx
2025-04-17 08:58:58,618 - INFO - Processing 1/288: Supriya Lifescience Limited (SUPRIYA)
2025-04-17 08:58:59,146 - INFO - Found valid NSE ticker: SUPRIYA.NS
2025-04-17 08:58:59,171 - ERROR - 
1 Failed download:
2025-04-17 08:58:59,171 - ERROR - ['SUPRIYA.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-08-17 -> 2021-09-16) (Yahoo error = "Data doesn\'t exist for startDate = 1629138600, endDate = 1631730600")')
2025-04-17 08:58:59,176 - WARNING - Data fetch failed for SUPRIYA.NS, retrying in 1.49 seconds...
2025-04-17 08:59:00,676 - ERROR - 
1 Failed download:
2025-04-17 08:59:00,676 - ERROR - ['SUPRIYA.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-08-17 -> 2021-09-16) (Yahoo error = "Data doesn\'t exist for startDate = 1629138600, endDate = 1631730600")')
2025-04-17 08:59:00,67

2025-04-17 08:59:29,027 - WARNING - Data fetch failed for SUPRIYA.BO, retrying in 2.99 seconds...
2025-04-17 08:59:32,027 - ERROR - 
1 Failed download:
2025-04-17 08:59:32,027 - ERROR - ['SUPRIYA.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-06-03 -> 2021-11-30) (Yahoo error = "Data doesn\'t exist for startDate = 1622658600, endDate = 1638210600")')
2025-04-17 08:59:33,774 - INFO - Found valid NSE ticker: SUPRIYA.NS
2025-04-17 08:59:35,215 - INFO - Waiting 1.61 seconds before next request...
2025-04-17 08:59:36,893 - INFO - Found valid NSE ticker: SUPRIYA.NS
2025-04-17 08:59:39,153 - INFO - Found valid NSE ticker: SUPRIYA.NS
2025-04-17 08:59:39,473 - INFO - Waiting 1.62 seconds before next request...
2025-04-17 08:59:41,174 - INFO - Found valid NSE ticker: SUPRIYA.NS
2025-04-17 08:59:42,840 - INFO - Found valid NSE ticker: SUPRIYA.NS
2025-04-17 08:59:43,282 - INFO - Waiting 3.30 seconds before next request...
2025-04-17 08:59:46,737 - INFO - Found valid N

2025-04-17 09:00:59,451 - ERROR - 
1 Failed download:
2025-04-17 09:00:59,451 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-07-03 -> 2021-10-31) (Yahoo error = "Data doesn\'t exist for startDate = 1625250600, endDate = 1635618600")')
2025-04-17 09:00:59,459 - WARNING - Data fetch failed for SURAJEST.BO, retrying in 1.30 seconds...
2025-04-17 09:01:00,772 - ERROR - 
1 Failed download:
2025-04-17 09:01:00,772 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-07-03 -> 2021-10-31) (Yahoo error = "Data doesn\'t exist for startDate = 1625250600, endDate = 1635618600")')
2025-04-17 09:01:00,777 - WARNING - Data fetch failed for SURAJEST.BO, retrying in 2.32 seconds...
2025-04-17 09:01:03,110 - ERROR - 
1 Failed download:
2025-04-17 09:01:03,110 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-07-03 -> 2021-10-31) (Yahoo error = "Data doesn\'t exi

2025-04-17 09:01:44,998 - WARNING - Data fetch failed for SURAJEST.BO, retrying in 2.55 seconds...
2025-04-17 09:01:47,559 - ERROR - 
1 Failed download:
2025-04-17 09:01:47,559 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-08-02 -> 2022-10-01) (Yahoo error = "Data doesn\'t exist for startDate = 1659378600, endDate = 1664562600")')
2025-04-17 09:01:49,448 - ERROR - 
1 Failed download:
2025-04-17 09:01:49,448 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-07-03 -> 2022-10-31) (Yahoo error = "Data doesn\'t exist for startDate = 1656786600, endDate = 1667154600")')
2025-04-17 09:01:49,459 - WARNING - Data fetch failed for SURAJEST.BO, retrying in 1.50 seconds...
2025-04-17 09:01:50,973 - ERROR - 
1 Failed download:
2025-04-17 09:01:50,973 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-07-03 -> 2022-10-31) (Yahoo error = "Data doesn\'t exi

2025-04-17 09:02:31,627 - WARNING - Data fetch failed for SURAJEST.BO, retrying in 1.14 seconds...
2025-04-17 09:02:32,782 - ERROR - 
1 Failed download:
2025-04-17 09:02:32,782 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-08-02 -> 2023-10-01) (Yahoo error = "Data doesn\'t exist for startDate = 1690914600, endDate = 1696098600")')
2025-04-17 09:02:32,787 - WARNING - Data fetch failed for SURAJEST.BO, retrying in 2.38 seconds...
2025-04-17 09:02:35,180 - ERROR - 
1 Failed download:
2025-04-17 09:02:35,180 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-08-02 -> 2023-10-01) (Yahoo error = "Data doesn\'t exist for startDate = 1690914600, endDate = 1696098600")')
2025-04-17 09:02:36,464 - ERROR - 
1 Failed download:
2025-04-17 09:02:36,464 - ERROR - ['SURAJEST.BO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-07-03 -> 2023-10-31) (Yahoo error = "Data doesn\'t exi

...

2025-04-17 13:01:58,392 - WARNING - Ticker lookup failed, retrying in 1.65 seconds...
2025-04-17 13:02:00,107 - ERROR - 
1 Failed download:
2025-04-17 13:02:00,107 - ERROR - ['ZOMATO.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:02:00,179 - ERROR - 
1 Failed download:
2025-04-17 13:02:00,179 - ERROR - ['ZOMATO.BO']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:02:05,484 - ERROR - 
1 Failed download:
2025-04-17 13:02:05,484 - ERROR - ['ZOMATO']: YFPricesMissingError('possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")')
2025-04-17 13:02:05,492 - WARNING - Ticker lookup failed, retrying in 2.06 seconds...
2025-04-17 13:02:07,661 - ERROR - 
1 Failed download:
2025-04-17 13:02:07,661 - ERROR - ['ZOMATO.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:02:07,848 - ERROR - 
1 Failed download:
2025-

2025-04-17 13:03:04,509 - ERROR - ['ZOMATO']: YFTzMissingError('possibly delisted; no timezone found')
2025-04-17 13:03:04,513 - WARNING - Data fetch failed for ZOMATO, retrying in 1.09 seconds...
2025-04-17 13:03:05,621 - ERROR - 
1 Failed download:
2025-04-17 13:03:05,621 - ERROR - ['ZOMATO']: YFTzMissingError('possibly delisted; no timezone found')
2025-04-17 13:03:05,625 - WARNING - Data fetch failed for ZOMATO, retrying in 2.17 seconds...
2025-04-17 13:03:07,809 - ERROR - 
1 Failed download:
2025-04-17 13:03:07,809 - ERROR - ['ZOMATO']: YFTzMissingError('possibly delisted; no timezone found')
2025-04-17 13:03:10,159 - ERROR - 
1 Failed download:
2025-04-17 13:03:10,159 - ERROR - ['ZOMATO.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:03:10,271 - ERROR - 
1 Failed download:
2025-04-17 13:03:10,271 - ERROR - ['ZOMATO.BO']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:03:12,958 - ERROR - 
1 

2025-04-17 13:04:37,675 - ERROR - 
1 Failed download:
2025-04-17 13:04:37,675 - ERROR - ['ZOMATO.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:04:37,796 - ERROR - 
1 Failed download:
2025-04-17 13:04:37,796 - ERROR - ['ZOMATO.BO']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:04:42,083 - ERROR - 
1 Failed download:
2025-04-17 13:04:42,083 - ERROR - ['ZOMATO']: YFPricesMissingError('possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")')
2025-04-17 13:04:42,091 - WARNING - Ticker lookup failed, retrying in 1.99 seconds...
2025-04-17 13:04:44,137 - ERROR - 
1 Failed download:
2025-04-17 13:04:44,137 - ERROR - ['ZOMATO.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:04:44,198 - ERROR - 
1 Failed download:
2025-04-17 13:04:44,199 - ERROR - ['ZOMATO.BO']: YFPricesMissingError('possibly delisted; n

2025-04-17 13:05:42,287 - ERROR - 
1 Failed download:
2025-04-17 13:05:42,287 - ERROR - ['ZOMATO.BO']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
2025-04-17 13:05:44,952 - ERROR - 
1 Failed download:
2025-04-17 13:05:44,952 - ERROR - ['ZOMATO']: YFPricesMissingError('possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")')
2025-04-17 13:05:44,961 - WARNING - Could not find valid exchange for ZOMATO, using as is
2025-04-17 13:05:47,420 - ERROR - 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZOMATO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZOMATO&crumb=xyYEEgvguRS
2025-04-17 13:05:47,420 - WARNING - Error getting financial statements (attempt 1): 'NoneType' object has no attribute 'update'
2025-04-17 13:05:51,423 - ERROR - 
1 Failed download:
2025-04-17 13:05

2025-04-17 13:07:53,107 - INFO - Found valid NSE ticker: ZUARIIND.NS
2025-04-17 13:07:55,838 - INFO - Found valid NSE ticker: ZUARIIND.NS
2025-04-17 13:07:57,504 - INFO - Waiting 3.27 seconds before next request...
2025-04-17 13:08:00,861 - INFO - Found valid NSE ticker: ZUARIIND.NS
2025-04-17 13:08:03,509 - INFO - Found valid NSE ticker: ZUARIIND.NS
2025-04-17 13:08:04,095 - INFO - Waiting 4.06 seconds before next request...
2025-04-17 13:08:08,291 - INFO - Found valid NSE ticker: ZUARIIND.NS
2025-04-17 13:08:11,131 - INFO - Found valid NSE ticker: ZUARIIND.NS
2025-04-17 13:08:11,694 - INFO - Waiting 3.19 seconds before next request...
2025-04-17 13:08:14,901 - INFO - Processing 287/288: Zydus Lifesciences Limited (ZYDUSLIFE)
2025-04-17 13:08:15,060 - INFO - Found valid NSE ticker: ZYDUSLIFE.NS
2025-04-17 13:08:17,567 - INFO - Found valid NSE ticker: ZYDUSLIFE.NS
2025-04-17 13:08:19,785 - INFO - Waiting 3.65 seconds before next request...
2025-04-17 13:08:23,515 - INFO - Found valid N